# AI as Judge

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM as a judge to evaluate LLM outputs. The evaluation can be based on any criteria. G-Eval is implemented by a library called [DeepEval](https://deepeval.com/) which includes a broader set of tests.


In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [3]:
from openai import OpenAI
import os

document_folder = "../../05_src/documents/"
blue_cross_file = "chesterton.txt"
file_path = os.path.join(document_folder, blue_cross_file)

with open(file_path, "r", encoding="utf-8") as f:
    blue_cross_text = f.read()

In [5]:
instructions = "You are an helpful assistant that summarizes works of fiction with a quirky and bubbly approach."
PROMPT = """
    Summarize the following story in at most four paragraphs. Please include all key characters and plot points.
    <story>
    {story}
    </story>
    In addition to the summary, add an introduction paragraph where you greet the reader and a conclusion where you share an opinion about the story.
"""

In [6]:
client = OpenAI()
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=1.2
)

In [7]:
response.output_text

"**Hello, fellow book lovers!** ✨ Are you ready to immerse yourself in a delightful blend of mystery and quirkiness? Welcome to the charmingly puzzling tales of *The Innocence of Father Brown* by G. K. Chesterton, where Father Brown, our endearing little detective priest, uses his sharp wits and compassion to unravel the murkiest of crimes! Now, let's unpack this marvelous collection, shall we?\n\nIn the opener, “The Blue Cross,” we meet Aristide Valentin, a clever French detective, who embarks on a quest to catch the infamous criminal Flambeau, only to hilariously discover that the scruffy priest he has overlooked—Father Brown—might just be the key to his capture. As their paths intertwine, the dark-and-dim Church juxtaposing the shenanigans of childhood conversations lead to deeper reflections on morality and humanity. Packing a punch with lively characters and shrewd observations, these early exchanges set the stage for both crime and curiosity!\n\nThrough eight more whimsy-laden ta

# Answer Relevancy

The answer relevancy metric evaluates how relevant the actual output of the LLM app is compared to the provided input. This metric is self-explaining in the sense that the output includes a reason for the metric score.

The metric is calculated as:

$$
AnswerRelevancy=\frac{NumberRelevantStatements}{TotalStatements}
$$

Reference: [Answer Relevancy](https://deepeval.com/docs/metrics-answer-relevancy). 

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-4o-mini",
    include_reason=True
)

test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)

In [ ]:
metric.measure(test_case)

In [ ]:
print(metric.score,metric.reason)

# Other Metrics

Other useful metric functions include:

+ [Faithfulness](https://deepeval.com/docs/metrics-faithfulness): evaluates whether the `actual_output` factually aligns with the contents of  `retrieval_context`. 
+ [Contextual Precision](https://deepeval.com/docs/metrics-contextual-precision): evaluates whether nodes in your `retrieval_context` that are relevant to the given input are ranked higher than irrelevant ones. 
+ [Contextual Recall](https://deepeval.com/docs/metrics-contextual-recall): evaluates the extent of which the retrieval_context aligns with the expected_output. 
+ [Contextual Relevancy](https://deepeval.com/docs/metrics-contextual-relevancy): evaluates the overall relevance of the information presented in your retrieval_context for a given input. 

# G-Eval

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM-as-a-judge with chain-of-thoughts (CoT) to evaluate LLM outputs based on ANY custom criteria. The G-Eval metric is the most versatile type of metric deepeval offers.

In [ ]:
instructions = "You are an helpful assistant that specializes in works of fiction."
PROMPT = """
    Based on the story below, answer the question provided.
    <story>
    {story}
    </story>
    <question>
    Who is the main antagonist in the story and what motivates their actions?
    </question>
"""

In [ ]:
client = OpenAI()
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=0.7
)

In [ ]:
response.output_text

## Evaluation Criteria

The most straightforward way to establish a metric is by using a single criteria.

In [ ]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the context.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

In [ ]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])

## Evaluation Steps 

G-Eval is flexible in many ways: notice that we can establish an evaluation criteria or a set of evaluation steps, that can help in guiding the model to follow specific steps to perform the evaluation.

In [ ]:
...

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'input'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are not OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

In [ ]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])